In [1]:
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [3]:
image_path=r'E:\data project\RPS Project'
categories=['rock','paper','scissor','none']
data=[]
CLASS_MAP = {
    "rock": 0,
    "paper": 1,
    "scissor": 2,
    "none": 3
}

NUM_CLASSES = len(CLASS_MAP)
def mapper(val):
    return CLASS_MAP[val]




In [4]:
for i in categories:
    path=os.path.join(image_path,i)
    label=categories.index(i)
    for img in os.listdir(path):
        pic=os.path.join(path,img)
        img_read=cv2.imread(pic)
        img_arr=cv2.resize(img_read,(100,100))
        data.append([img_arr,label])
        
random.shuffle(data)   
len(data)


1200

In [9]:
data[432]

[array([[[247, 253, 255],
         [254, 254, 252],
         [255, 255, 251],
         ...,
         [255, 255, 250],
         [255, 255, 252],
         [254, 254, 253]],
 
        [[255, 254, 253],
         [189, 177, 170],
         [193, 179, 169],
         ...,
         [197, 187, 181],
         [195, 188, 183],
         [226, 223, 219]],
 
        [[255, 254, 250],
         [191, 179, 169],
         [195, 179, 166],
         ...,
         [203, 186, 179],
         [199, 186, 179],
         [226, 221, 217]],
 
        ...,
 
        [[255, 253, 254],
         [137, 130, 131],
         [140, 129, 131],
         ...,
         [142, 131, 133],
         [142, 132, 134],
         [197, 193, 194]],
 
        [[255, 254, 254],
         [135, 129, 130],
         [136, 128, 129],
         ...,
         [141, 133, 134],
         [140, 133, 134],
         [196, 193, 194]],
 
        [[255, 255, 255],
         [193, 190, 191],
         [194, 190, 191],
         ...,
         [197, 194, 195],
  

In [10]:
x=[]
y=[]
for feature,label in data:
    x.append(feature)
    y.append(label)
    
x=np.array(x)
y=np.array(y)
x=x/255

In [11]:
import pickle
pickle.dump(x,open('x.pkl','wb'))
pickle.dump(y,open('y.pkl','wb'))

In [12]:
model=keras.Sequential([
keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=x.shape[1:]),
keras.layers.MaxPooling2D(2,2),
keras.layers.Conv2D(32,(3,3),activation='relu'),
keras.layers.MaxPooling2D(2,2),
keras.layers.Conv2D(16,(3,3),activation='relu'),
keras.layers.MaxPooling2D(2,2), 
keras.layers.Flatten(),
keras.layers.Dense(128,activation='relu'),
keras.layers.Dense(64,activation='relu'),
keras.layers.Dense(4,activation='softmax'),  
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=10,validation_split=0.1)

Epoch 1/10
34/34 [==============================] - 11s 326ms/step - loss: 1.0061 - accuracy: 0.5741 - val_loss: 0.8627 - val_accuracy: 0.6083
Epoch 2/10
34/34 [==============================] - 11s 325ms/step - loss: 0.4024 - accuracy: 0.8454 - val_loss: 0.2654 - val_accuracy: 0.8750
Epoch 3/10
34/34 [==============================] - 12s 341ms/step - loss: 0.1744 - accuracy: 0.9444 - val_loss: 0.1514 - val_accuracy: 0.9333
Epoch 4/10
34/34 [==============================] - 12s 359ms/step - loss: 0.0930 - accuracy: 0.9750 - val_loss: 0.0709 - val_accuracy: 0.9917
Epoch 5/10
34/34 [==============================] - 16s 478ms/step - loss: 0.0571 - accuracy: 0.9843 - val_loss: 0.0415 - val_accuracy: 0.9917
Epoch 6/10
34/34 [==============================] - 15s 453ms/step - loss: 0.0419 - accuracy: 0.9889 - val_loss: 0.0395 - val_accuracy: 0.9917
Epoch 7/10
34/34 [==============================] - 16s 456ms/step - loss: 0.0254 - accuracy: 0.9935 - val_loss: 0.0635 - val_accuracy: 0.9917

In [17]:
model.save("rock-paper-scissors-model")

INFO:tensorflow:Assets written to: rock-paper-scissors-model\assets


In [15]:
Class_map={
    0:'rock',
    1:'paper',
    2:'scissor',
    3:'none'
}
def Class(val):
    return Class_map[val]

In [16]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    cv2.rectangle(frame,(100,100),(500,500),(255,255,255),2)
    img=frame[100:500, 100:500]
    img_new=cv2.resize(img,(100,100))
    pre=model.predict(np.array([img_new]))
    pre_class=np.argmax(pre[0])
    result=Class(pre_class)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Your Move: " + result,(50, 50), font, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('rps',frame)
    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()